In [ ]:
!python --version  #파이썬 버전 확인

Python 3.10.12


In [ ]:
# 필요한 패키지 설치
!apt-get update
!apt-get install -y wget unzip

# 크롬 브라우저 설치
!wget -O /tmp/chrome-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.76/linux64/chrome-linux64.zip
!unzip -o /tmp/chrome-linux64.zip -d /usr/local/
!ln -sf /usr/local/chrome-linux64/chrome /usr/bin/google-chrome  # 심볼릭 링크 강제로 덮어쓰기

# 크롬드라이버 설치
!wget -O /tmp/chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.76/linux64/chromedriver-linux64.zip
!unzip -o /tmp/chromedriver.zip -d /usr/local/bin/
!mv /usr/local/bin/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
!chmod +x /usr/local/bin/chromedriver
!pip install selenium


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Fetched 3,733 kB in 4s (961 kB/s)
Reading package lists... Done

In [ ]:
import selenium
print(selenium.__version__)

4.21.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os


# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")  # GUI 없는 환경에서 사용
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--remote-debugging-port=9222')

# 여기서 Chromium 브라우저의 경로를 명시적으로 설정합니다.
options.binary_location = '/usr/local/chrome-linux64/chrome'


# ChromeDriver 경로 설정
chrome_driver_path = '/usr/local/bin/chromedriver'  # ChromeDriver 경로

# ChromeDriver 서비스 설정
service = Service(chrome_driver_path)

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 장학 공고 페이지 접속
driver.get("https://suwon4u.or.kr/?p=21")

# 공고 목록 가져오기
post_links = driver.find_elements(By.CSS_SELECTOR, "tbody tr")


# 각 게시물에 접속하여 첨부 파일 다운로드
for index, row in enumerate(post_links):
    # 다시 요소를 찾음
    post_links = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    row = post_links[index]
    link = row.find_element(By.CSS_SELECTOR, "td.txt_left a")

    # 링크 클릭
    driver.execute_script("arguments[0].click();", link)

    # 페이지가 로드될 때까지 잠시 기다림
    time.sleep(2)

    try :
      # 장학금 종류
      scholarship_type = driver.find_element(By.XPATH, '//tr[th[text()="장학금종류"]]/td').text

      # 선발시기
      selection_period = driver.find_element(By.XPATH, '//tr[th[text()="선발시기"]]/td').text

      # 장학금액
      scholarship_amount = driver.find_element(By.XPATH, '//tr[th[text()="장학금액"]]/td').text

      # 신청자격
      eligibility = driver.find_element(By.XPATH, '//tr[th[text()="신청자격"]]/td').text

      # 정보 출력
      print(f"장학금 종류: {scholarship_type}")
      print(f"선발시기: \n{selection_period}")
      print(f"장학금액: {scholarship_amount}")
      print(f"신청자격: \n{eligibility}\n")
      print("-------------------------------------------------------")
    except Exception as e :
      print(f"Error extracting information: {e}")

    # 뒤로 가기
    driver.back()
    time.sleep(3)  # 페이지가 로드될 때까지 잠시 기다림

# 웹 드라이버 종료
driver.quit()

장학금 종류: 효·선행장학금
선발시기: 
상반기 (3월)
※ 추후 일정이 변경될 수 있음
장학금액: 연간 최대 400만원
(학교 및 각종 기관에서 지원받은 장학금을 차감하고 차액을 지급, 방송통신대학 및 사이버대학의 경우 등록금 고지서 금액 기준)
신청자격: 
● 공고일 현재 계속해서 2년 이상 수원시에 주민등록을 두고 거주한 학생
(단, 재학 중인 학교에 소재한 지역에 주소를 둔 경우 부모님의 거주기간 인정 - 학생 초본, 부 또는 모 초본, 가족관계증명서 첨부)

● 효 또는 선행을 실천하여 타의 귀감이 되는 학생

※ 자세한 내용은 첨부파일 참고

-------------------------------------------------------
장학금 종류: 우수장학금
선발시기: 
상반기 (3월)
※ 추후 일정이 변경될 수 있음
장학금액: 연간 최대 400만원
(학교 및 각종 기관에서 지원받은 장학금을 차감하고 차액을 지급, 방송통신대학 및 사이버대학의 경우 등록금 고지서 금액 기준)
신청자격: 
● 공고일 현재 계속해서 2년 이상 수원시에 주민등록을 두고 거주한 학생
(단, 재학 중인 학교에 소재한 지역에 주소를 둔 경우 부모님의 거주기간 인정 - 학생 초본, 부 또는 모 초본, 가족관계증명서 첨부)

● 국내 대학교에 재학 중인 학업성적이 우수한 학생

※ 자세한 내용은 첨부파일 참고

-------------------------------------------------------
장학금 종류: 희망장학금
선발시기: 
상반기 (3월)
※ 추후 일정이 변경될 수 있음
장학금액: 연간 최대 400만원
(학교 및 각종 기관에서 지원받은 장학금을 차감하고 차액을 지급, 방송통신대학 및 사이버대학의 경우 등록금 고지서 금액 기준)
신청자격: 
● 공고일 현재 계속해서 2년 이상 수원시에 주민등록을 두고 거주한 학생
(단, 재학 중인 학교에 소재한 지역에 주소를 둔 경우 부모님의 거주기간 인정 - 학생 초본, 부 또는 모 초본, 가족관계증명서 첨부)

● 